In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
import math

import DDS_CMpack as CM

%matplotlib inline

data_dir = '/Users/dengdingshan/Documents/GitHub/dds_python/Summer2018/data/'

# Step1 解t0时刻的位置和速度

In [76]:
position_fram = pd.read_csv(data_dir + 'Echo1st.csv')
observe_times = len(position_fram)

In [77]:
position_fram

,UT1_hours,alpha,delta,year,month,days
0,21.575128,142.935000,8.521111,1965,1,14
1,21.603111,157.274167,-2.395000,1965,1,14
2,21.631420,171.817917,-14.508333,1965,1,14
3,21.654491,183.134167,-23.563333,1965,1,14
4,21.714094,208.640417,-40.020278,1965,1,14
5,21.742876,219.102083,-44.998611,1965,1,14


In [78]:
# 常数表
R_earth = 6371
au = 149597870
r_station_earth = 0.999102 #*R_earth
r_earth_sun = 1*au

time_unit = 806.81163 #806.8116 # SI 

# 测站的地理位置
lam = 118.82091666 # degrees
phi = 31.893611111 # degrees

# headers index: UT1_hours  alpha  delta  year  month  days

UT1_hours = position_fram['UT1_hours']
alpha = position_fram['alpha']
delta = position_fram['delta']
year = position_fram['year']
month = position_fram['month']
days = position_fram['days']


In [79]:
# 进行拉普拉斯方法所需要的星表归算

P = []
Q = []
Lambda_all = []
Niu_all = []
Miu_all = []

for i in range(observe_times):   
    # 计算观测数据的儒略日时间和恒星时
    SG_s,jd_s = CM.UTC2SG(year[i],month[i],days[i],UT1_hours[i])
    SG_degree = SG_s%(3600*24)/(3600*24)*360
    
    # 通过每一个恒星时去归算出测站到地心的RA,DEC
    RA_station = SG_degree + lam
    DEC_station = phi
    station_earth = CM.RADEC2xyz(r_station_earth,RA_station,DEC_station)
    
    X,Y,Z = station_earth
    
    # 再根据测站到地心的位置最终酸楚Pj,Qj
    delta_pi = delta[i]*2*np.pi/360
    alpha_pi = alpha[i]*2*np.pi/360
    
    Lambda = np.cos(delta_pi)*np.cos(alpha_pi)
    Miu = np.cos(delta_pi)*np.sin(alpha_pi)
    Niu = np.sin(delta_pi)
    Lambda_all.append(Lambda)
    Miu_all.append(Miu)
    Niu_all.append(Niu)
    
    P_j = Niu*X - Lambda*Z
    Q_j = Niu*Y - Miu*Z
    
    P.append(P_j);Q.append(Q_j)

PF_fill = position_fram
PF_fill['P'] = np.array(P)
PF_fill['Q'] = np.array(Q)
PF_fill['Lambda'] = np.array(Lambda_all)
PF_fill['Miu'] = np.array(Miu_all)
PF_fill['Niu'] = np.array(Niu_all)


In [80]:
PF_fill

,UT1_hours,alpha,delta,year,month,days,P,Q,Lambda,Miu,Niu
0,21.575128,142.935000,8.521111,1965,1,14,0.296272,-0.351086,-0.789144,0.596067,0.148174
1,21.603111,157.274167,-2.395000,1965,1,14,0.520311,-0.193223,-0.921558,0.385984,-0.041788
2,21.631420,171.817917,-14.508333,1965,1,14,0.708283,-0.008122,-0.958257,0.137781,-0.250521
3,21.654491,183.134167,-23.563333,1965,1,14,0.805554,0.131507,-0.915248,-0.050115,-0.399763
4,21.714094,208.640417,-40.020278,1965,1,14,0.870732,0.370844,-0.672115,-0.367065,-0.643059
5,21.742876,219.102083,-44.998611,1965,1,14,0.855495,0.434417,-0.548745,-0.445986,-0.707090


In [81]:
# t0的值要预先给定，这个值在这种写法底下使用24小时的单位值（粗略的换算）
t0 = PF_fill['UT1_hours'].sum()/6

# 给定所需要的按照理论单位的时间值
T = (PF_fill['UT1_hours'] - t0)*3600/time_unit
F0 = np.ones(observe_times).tolist()
G0 = T.tolist()


In [82]:
G

[-0.224927164225417, 0.0043345582413262425, 0.39870543760012317]

In [83]:
# 选定用于归算轨道六根数的数据点（观测序数）
# Ob_index = [i for i in range(6)]
Ob_index = [1,3,5]

############################################

# Laplace循环长数值和初值
F = [ F0[i] for i in Ob_index ]
G = [ G0[i] for i in Ob_index ]
Niu = Niu_all
Miu = Miu_all
Lambda = Lambda_all

# Laplace循环
Nwind = 0
key = 0
while key == 0:
    A = []
    b = []
    nwind = 0
    # 构造系数矩阵和常数阵
    for j in Ob_index:
         
        A.append([Niu[j]*F[nwind],0,-Lambda[j]*F[nwind],Niu[j]*G[nwind],0,-Lambda[j]*G[nwind]])
        A.append([0,Niu[j]*F[nwind],-Miu[j]*F[nwind],0,Niu[j]*G[nwind],-Miu[j]*G[nwind]])
        b.append(P[j])
        b.append(Q[j])
        nwind += 1
    
    AA = np.array(A)
    b = np.array(b)
    
    # 解出r0和V0
    ans = np.linalg.solve(np.dot(AA.T.copy(),AA),np.dot(AA.T.copy(),b))
    r0 = ans[:3]
    v0 = ans[3:]
        
    # 用r0和v0求出新的F和G
    norm_r0 = np.sqrt(np.dot(r0,r0))
    norm_v0 = np.sqrt(np.dot(v0,v0))
    
    F_old = F; G_old = G
    F = [];G = []
    for k in Ob_index:
        F.append(CM.F(norm_r0,norm_v0,T[k]))
        G.append(CM.G(norm_r0,norm_v0,T[k]))
    
    # 上述循环完成，计数一次
    Nwind += 1
    
    # 判断是否跳出循环
    delta_F = np.abs(np.array(F) - np.array(F_old))
    delta_G = np.abs(np.array(G) - np.array(G_old))
    biggest = np.max([np.max(delta_F),np.max(delta_G)])
    if biggest < 1e-13:
        print('end: ',int(Nwind))
        print(r0,v0)
        key = 1

end:  77
[-1.12749382 -0.27757062  0.39017866] [-0.02214238 -0.64540905 -0.58476384]


In [84]:
print(r0*R_earth,norm_r0*R_earth,(norm_r0-r_station_earth)*R_earth)

print(v0*R_earth/time_unit,norm_v0*R_earth/time_unit)

print(t0*3600)

[-7183.26313001 -1768.40243814  2485.82825775] 7804.220557185129 1438.941715185129
[-0.17484762 -5.0964821  -4.61759632] 6.879454721223272
77952.67183199999


# Step 2 解轨道六根数

In [85]:
# 取轨国际单位制的常数表

R_earth = 6371e3 # m

M_earth = 5.965e24 # kg
G_graviation = 6.672e-11 # N·m^2 /kg^2 

au = 149597870e3 # m
r_station_earth = 0.999102 #*R_earth
r_earth_sun = 1*au

# time_unit = 806.81163 #806.8116 # SI 

# miu_GM = 398600.5e-6 # km^3/SI^2

miu_GM = G_graviation*M_earth # m^3/SI^2

In [86]:
rg = r0*R_earth
norm_rg = np.sqrt(np.dot(rg,rg))
vg = v0*R_earth/time_unit
norm_vg = np.sqrt(np.dot(vg,vg))
tg = t0*time_unit

In [87]:
a = 1/(2/norm_rg - norm_vg**2/miu_GM)

In [88]:
n = np.sqrt(miu_GM/(a**3))

In [89]:
e = np.sqrt( (1 - norm_rg/a)**2 + (np.dot(rg,vg) / (n*a**2))**2)

In [90]:
cos_E = (1 - norm_rg/a)/e
sin_E = (np.dot(rg,vg) / (a**2*n))/e
E = math.atan2(sin_E,cos_E)

In [91]:
M = E - e*math.sin(E)

In [128]:
P = (cos_E/norm_rg)*rg - (sin_E/(a*n))*vg
Q = (sin_E/(norm_rg)*np.sqrt(1-e**2))*rg + ((cos_E - e)/(a*n*np.sqrt(1-e**2)))*vg
R = np.cross(P,Q)
P,Q

(array([ 0.87150656,  0.01073892, -0.49026624]),
 array([0.29811113, 0.77931308, 0.55032816]))

In [93]:
pz = P[2]; qz = Q[2]
tan_w_raw = pz/qz
if pz >=0 and qz >= 0:
    # w I
    w = math.atan(tan_w_raw)
elif pz >= 0 and qz < 0:
    # w II
    w = math.atan(tan_w_raw + np.pi)
elif pz < 0 and qz < 0:
    # w III
    w = math.atan(tan_w_raw + np.pi)
elif pz < 0 and qz >= 0:
    # w IV
    w = math.atan(2*np.pi - tan_w_raw)

In [94]:
h = np.cross(rg,vg)
norm_h = np.sqrt(np.dot(h,h))

h_A, h_B, h_C = h[0:3]

# get cos_i, sin_i
cos_i = (h_C/norm_h)
tan_i = np.sqrt(h_A**2 + h_B**2) / h_C

sin_i = tan_i*cos_i

# get cos_\sin_Omega
sin_Omega = h_A/(norm_h*sin_i)
cos_Omega = -h_B/(norm_h*sin_i)

i = math.atan2(sin_i,cos_i)
Omega = math.atan2(sin_Omega,cos_Omega)

In [95]:
print(a/R_earth,e,M,i,Omega,w)

1.1427392574526176 0.07538000853508695 -2.816315224978294 0.8280766091331286 0.555000809691323 1.4322973968788328


In [96]:
np.rad2deg(i)

47.44529481683259

# Step 3 反算星历表


In [101]:
t3 = t0
T3 = (PF_fill['UT1_hours'] - t3)*3600/24

In [116]:
M_total = M + n*T3

In [117]:
# 迭代 开普勒方程解 E
E_total = []
for i in range(len(Mt)):
    epsilon = 1e-20
    key = 0
    En0 = M_total[i]
    
    En_new = En0
    while key == 0:
        En_old = En_new
        En_new = M_total[i] + e*np.sin(En_old)
        delta_En = np.abs(En_new - En_old)

        if delta_En < epsilon:
            key = 1;

    E_total.append(En_new)

In [129]:
# 计算新的 P,Q
P_new = np.array([np.cos(Omega)*np.cos(w) - np.sin(Omega)*np.sin(w)*np.cos(i),\
                 np.sin(Omega)*np.cos(w) + np.cos(Omega)*np.sin(w)*np.cos(i),\
                 np.sin(w)*np.sin(i) ])
Q_new = np.array([-np.cos(Omega)*np.sin(w) - np.sin(Omega)*np.cos(w)*np.cos(i),\
                 -np.sin(Omega)*np.sin(w) + np.cos(Omega)*np.cos(w)*np.cos(i),\
                 np.cos(w)*np.sin(i)])

### 新算出来的P_new,Q_new有误！
P_new,Q_new

(array([-0.03070849,  0.31152404, -0.94974195]),
 array([-0.86239754, -0.48861485, -0.1323858 ]))

In [125]:
# 计算新的 r,v
RV = []
for Et in E_total:
    rt = a*(np.cos(Et) - e)*P_new + a*np.sqrt(1 - e**2)*np.sin(Et)*Q_new
    rt_norm = np.sqrt( np.dot(rt,rt) )

    vt = -a**2*n/rt_norm*np.sin(Et)*P_new + a**2*n/rt_norm*np.sqrt(1 - e**2)*np.cos(Et)*Q_new
    vt_norm = np.sqrt( np.dot(vt,vt) )

    L_rt = rt/ np.sqrt( np.dot(rt,rt))

    print(rt_norm/10**3,vt_norm/10**3)
    
    RV.append([rt,vt])

7806.010854468222 6.877754393602712
7805.37932030171 6.878354149565239
7804.731909615432 6.878969029614856
7804.197960855345 6.879476184295364
7802.7923478627445 6.880811415294666
7802.10007522515 6.881469105956507


In [126]:
# 归算最终的赤经赤纬
P_earth = rt
station_earth = np.array([-0.81000791, -0.25188507,  0.52786921])*R_earth

P_station = P_earth - station_earth

L_forecast = P_station/np.sqrt(np.dot(P_station,P_station))

L_forecast

array([0.87549008, 0.04517822, 0.48111958])

In [127]:
PF_fill

,UT1_hours,alpha,delta,year,month,days,P,Q,Lambda,Miu,Niu
0,21.575128,142.935000,8.521111,1965,1,14,0.296272,-0.351086,-0.789144,0.596067,0.148174
1,21.603111,157.274167,-2.395000,1965,1,14,0.520311,-0.193223,-0.921558,0.385984,-0.041788
2,21.631420,171.817917,-14.508333,1965,1,14,0.708283,-0.008122,-0.958257,0.137781,-0.250521
3,21.654491,183.134167,-23.563333,1965,1,14,0.805554,0.131507,-0.915248,-0.050115,-0.399763
4,21.714094,208.640417,-40.020278,1965,1,14,0.870732,0.370844,-0.672115,-0.367065,-0.643059
5,21.742876,219.102083,-44.998611,1965,1,14,0.855495,0.434417,-0.548745,-0.445986,-0.707090
